In [1]:
import os
os.chdir("..")

from wumpusWorld.agent.NaiveAgent import *
from wumpusWorld.environment.Environment import *
from wumpusWorld.environment.Percept import *
from wumpusWorld.environment.Agent import *
from wumpusWorld.environment.Action import *

In [5]:
i=0

initialEnv = Environment.create()
agent = NaiveAgent.create()
percept = Percept.create(initialEnv.isStench(Coords(0,0)), 
                         initialEnv.isBreeze(Coords(0,0)), 
                         initialEnv.isGlitter(Coords(0,0)))
print("-----INITIAL STATE-----")

while not initialEnv.terminated:
    initialEnv.visualize(agent)
    percept.show()
    agent.show()
    print("\n -----STEP " + str(i) + "-----")
    action = NaiveAgent.nextAction()
    percept, agent = initialEnv.applyAction(percept, action, agent)
    i+=1
percept.show()
agent.show()

-----INITIAL STATE-----
    | P  |    |   W|
    |    |  G |    |
    |    | P  | P  |
A   | P  |    |    |

stench: False, breeze: True, glitter: False, bump: False, scream: False, isTerminated: False, reward: 0
arrow: True, gold: False, alive: True

 -----STEP 0-----
Grab
    | P  |    |   W|
    |    |  G |    |
    |    | P  | P  |
A   | P  |    |    |

stench: False, breeze: True, glitter: False, bump: False, scream: False, isTerminated: False, reward: -1
arrow: True, gold: False, alive: True

 -----STEP 1-----
Forward
stench: False, breeze: False, glitter: False, bump: False, scream: False, isTerminated: True, reward: -1002
arrow: True, gold: False, alive: False


In [ ]:
import random
import copy

In [ ]:
action_pt = -1
climb_w_gold = 1000
death_pt = -1000
arrow_pt = -10
total_pt = 0

In [ ]:
class Coords():
  '''
  This class is for storing x, y coordinates of WumpusWorld game board
  '''  
  def __init__(self, x, y):
    self.x = x
    self.y = y
    
  def __eq__(self, coords):
    '''
    Checks whether two coordinates point to the same place on the board
    '''
    return coords.x == self.x and coords.y == self.y

In [ ]:
class Action():
  '''
  Action class has 6 children classes:
  Forward, TurnLeft, TurnRight, Grab, Climb, Shoot
  These are all the moves an agent can make
  It takes environment, percept, agent objects, and returns them with updated states
  '''   
  @staticmethod
  def __call__(environment, percept, agent):
    pass

class Forward(Action):
  '''
  Forward action does the following:
  Calls agent.forward() method to move the agent forward
  Checks if the agent has moved into a pit or wumpus location - death
  Checks if the agent has moved into a gold location
  Returns updated environment, percept, agent objects
  '''
  @staticmethod
  def __call__(environment, percept, agent):
    agent.forward(environment.gridWidth, environment.gridHeight)
    death = (environment.isWumpusAt(agent.location) and environment.wumpusAlive) \
             or environment.isPitAt(agent.location)
    agent.isAlive =  not death
    environment.goldLocation = agent.location if agent.hasGold \
                               else environment.goldLocation
    environment.terminated = death
    
    return environment, Percept(environment.isStench(agent.location), 
                                environment.isBreeze(agent.location), 
                                environment.isGlitter(agent.location), 
                                agent.hasBumped, 
                                False, 
                                not agent.isAlive, 
                                percept.reward + action_pt if (agent.isAlive) \
                                  else percept.reward + action_pt + death_pt) \
                        ,agent

class TurnLeft(Action):
  '''
  TurnLeft action calls agent.turnLeft() method
  Returns updated environment, percept, agent objects
  '''
  @staticmethod
  def __call__(environment, percept, agent):
    agent.turnLeft()
    return environment, Percept(environment.isStench(agent.location), 
                          environment.isBreeze(agent.location), 
                          environment.isGlitter(agent.location), 
                          False, 
                          False, 
                          False, 
                          percept.reward + action_pt) \
                        ,agent

class TurnRight(Action):
  '''
  TurnLeft action calls agent.turnRight() method
  Returns updated environment, percept, agent objects
  '''
  @staticmethod
  def __call__(environment, percept, agent):
    agent.turnRight()
    return environment, Percept(environment.isStench(agent.location), 
                          environment.isBreeze(agent.location), 
                          environment.isGlitter(agent.location), 
                          False, 
                          False, 
                          False, 
                          percept.reward + action_pt) \
                        ,agent
  
class Grab(Action):
  '''
  If the location has Glitter, agent now has gold
  Returns updated environment, percept, agent objects
  '''
  @staticmethod
  def __call__(environment, percept, agent):
    agent.hasGold = environment.isGlitter(agent.location)
    environment.goldLocation = agent.location if agent.hasGold \
                               else environment.goldLocation
    return environment, Percept(environment.isStench(agent.location), 
                        environment.isBreeze(agent.location), 
                        environment.isGlitter(agent.location), 
                        False, 
                        False, 
                        False, 
                        percept.reward + action_pt) \
                        ,agent


class Climb(Action):
  '''
  The agent can only climb if it's in start location
  Also checks if climbing without gold is allowed
  Returns updated environment, percept, agent objects
  '''  
  @staticmethod
  def __call__(environment, percept, agent):
    inStartLocation = agent.location == Coords(0,0)
    success = agent.hasGold and inStartLocation
    isTerminated = success or (environment.allowClimbWithoutGold and inStartLocation)
    environment.terminated = isTerminated
    
    return environment, Percept(False, 
                          False, 
                          environment.isGlitter(agent.location), 
                          False, 
                          False, 
                          isTerminated, 
                          percept.reward + action_pt + climb_w_gold if success else percept.reward + action_pt) \
                        ,agent

class Shoot(Action):
  '''
  Calls environment.killAttemptSuccessful() to check if wumpus was on line of fire
  Updates the agent to have no arrows
  Returns updated environment, percept, agent objects
  '''  
  @staticmethod
  def __call__(environment, percept, agent):
    hadArrow = copy.copy(agent.hasArrow)
    wumpusKilled = environment.killAttemptSuccessful(agent)
    agent.hasArrow = False
    environment.wumpusAlive = environment.wumpusAlive and not wumpusKilled

    return environment, Percept(environment.isStench(agent.location), 
                          environment.isBreeze(agent.location), 
                          environment.isGlitter(agent.location), 
                          False, 
                          wumpusKilled, 
                          False, 
                          percept.reward + action_pt + arrow_pt if hadArrow else percept.reward + action_pt) \
                        ,agent

In [ ]:
class Orientation():
  '''
  This class keeps track of agent's orientation
  Has a child class -> Agent
  If turnLeft -> switch into the previous orientation in the list
  If turnRight -> switch into the next orientation in the list
  '''
  def __init__(self, orientation = "East"):
    self.orient_list = ["North", "East", "South", "West", "North", "West"]
    self.orientation = orientation

  def turnLeft(self):
    ind = self.orient_list.index(self.orientation)
    self.orientation = self.orient_list[ind-1]

  def turnRight(self):
    ind = self.orient_list.index(self.orientation)
    self.orientation = self.orient_list[ind+1]

In [ ]:
class Agent(Orientation):
    '''
    Keeps track of Agent's state within the environment - hidden from the agent itself
    
    '''
    def __init__(self, 
                 orientation = "East",
                 location = Coords(0, 0),
                 hasBumped = False,
                 hasGold = False,
                 hasArrow = True,
                 isAlive = True):
      super().__init__()
      self.location = location
      self.hasBumped = hasBumped
      self.hasGold = hasGold
      self.hasArrow = hasArrow
      self.isAlive = isAlive
    
    def forward(self, gridWidth: int, gridHeight: int):
      '''
      Depending on the orientation and initial location, move the agent into the next cell
      If the agent hits the wall, the agent hasBumped
      '''  
      prev_location = copy.copy(self.location)

      if self.orientation == "West":
        self.location = Coords(max(0, self.location.x - 1), self.location.y)
      elif self.orientation == "East":
        self.location = Coords(min(gridWidth - 1, self.location.x + 1), self.location.y)
      elif self.orientation == "South":
        self.location = Coords(self.location.x, max(0, self.location.y - 1), )
      elif self.orientation == "North":
        self.location = Coords(self.location.x, min(gridHeight - 1, self.location.y + 1), )

      self.hasBumped = prev_location == self.location

In [ ]:
class Percept():
  '''
  This class keeps track of agent's perceptions
  '''  
  def __init__(self,
               stench: bool, 
               breeze: bool, 
               glitter: bool, 
               bump: bool, 
               scream: bool, 
               isTerminated: bool, 
               reward: int):
    self.stench = stench
    self.breeze = breeze
    self.glitter = glitter
    self.bump = bump
    self.scream = scream
    self.isTerminated = isTerminated
    self.reward = reward

  @staticmethod  
  def create(stench: bool, 
             breeze: bool, 
             glitter: bool):
    return Percept(stench, breeze, glitter, False, False, False, 0)

  def show(self):
    print("stench: ", self.stench, ", breeze: ", self.breeze, ", glitter: ", self.glitter, ", bump: ", self.bump,
          ", scream: ", self.scream, ", isTerminated: ", self.isTerminated, ", reward: ", self.reward, sep="")

In [ ]:
class Environment():
  '''
  This class builds the board: generates a random wumpus location, gold location,
  pit locations, and keeps track of the state of the game
  '''  
  def __init__(self,
               pitLocations: list,
               wumpusLocation: Coords,
               goldLocation: Coords,
               gridWidth: int = 4,
               gridHeight: int = 4,
               pitProb: float = 0.2,
               allowClimbWithoutGold: bool = True,
               terminated: bool = False,
               wumpusAlive: bool = True):
    
    self.gridWidth = gridWidth
    self.gridHeight = gridHeight
    self.pitProb = pitProb
    self.allowClimbWithoutGold = allowClimbWithoutGold
    self.pitLocations = pitLocations
    self.terminated = terminated
    self.wumpusLocation = wumpusLocation
    self.wumpusAlive = wumpusAlive
    self.goldLocation = goldLocation

  def isPitAt(self, coords : Coords):
    return coords in self.pitLocations

  def isWumpusAt(self, coords: Coords):
    return self.wumpusLocation == coords

  def isAgentAt(self, agent: Agent, coords: Coords):
    return agent.location == coords

  def isGlitter(self, coords: Coords):
    return self.goldLocation == coords

  def isGoldAt(self, coords: Coords):
    return self.goldLocation == coords

  def killAttemptSuccessful(self, agent: Agent):
    '''
    Depending on the agent's location and orientation, check if the wumpus in in line of fire
    '''    
    if agent.orientation == "West":
      wumpusInLineOfFire = agent.location.x > self.wumpusLocation.x and agent.location.y == self.wumpusLocation.y
    elif agent.orientation == "East":
      wumpusInLineOfFire = agent.location.x < self.wumpusLocation.x and agent.location.y == self.wumpusLocation.y
    elif agent.orientation == "South":
      wumpusInLineOfFire = agent.location.x == self.wumpusLocation.x and agent.location.y > self.wumpusLocation.y
    elif agent.orientation == "North":
      wumpusInLineOfFire = agent.location.x == self.wumpusLocation.x and agent.location.y < self.wumpusLocation.y

    return wumpusInLineOfFire and agent.hasArrow and self.wumpusAlive

  def adjacentCells(self, coords: Coords):
    '''
    Given certain x,y coordinates, return a list of all adjacent cells (no diagonal cells)
    '''    
    toLeft = [Coords(coords.x - 1, coords.y)] if coords.x > 0 else []
    toRight = [Coords(coords.x + 1, coords.y)] if (coords.x < self.gridWidth - 1) else []
    below = [Coords(coords.x, coords.y - 1)] if (coords.y > 0) else []
    above = [Coords(coords.x, coords.y + 1)] if coords.y < self.gridHeight - 1 else []
    return toLeft + toRight + below + above

  def isPitAdjacent(self, coords: Coords):
    return any(cell in self.pitLocations for cell in self.adjacentCells(coords))

  def isWumpusAdjacent(self, coords: Coords):
    return self.wumpusLocation in self.adjacentCells(coords)

  def isBreeze(self, coords: Coords):
    return self.isPitAdjacent(coords)

  def isStench(self, coords: Coords):
    return self.isWumpusAdjacent(coords) or self.isWumpusAt(coords)
    
  def applyAction(self, percept: Percept, action: Action, agent: Agent):
    '''
    Lets the agent act only if the game is not terminated
    '''
    if self.terminated:
      percept = Percept(False, False, False, False, False, True, 0)
    else:
      self, percept, agent = action(self, percept, agent)
    return percept, agent

  def visualize(self, agent: Agent):
    '''
    Simple visualization of the game - show where are the pits, gold, wumpus and the agent
    If the wumpus is alive, show capital "W", if not, lower case "w"
    '''    
    wumpusSymbol = "W" if self.wumpusAlive else "w"
    s = ""
    for j in range(self.gridHeight-1, -1, -1):
      for i in range(self.gridWidth):
        s = s + "A" if self.isAgentAt(agent, Coords(i,j)) else s + " "
        s = s + "P" if self.isPitAt(Coords(i,j)) else s + " "
        s = s + "G" if self.isGoldAt(Coords(i,j)) else s + " "
        s = s + wumpusSymbol if self.isWumpusAt(Coords(i,j)) else s + " "
        s += "|"
      s += "\n"
    print(s)
  
  '''
  initialize the environment
  '''
  @staticmethod
  def create(gridWidth: int = 4,
             gridHeight: int = 4,
             pitProb: float = 0.2,
             allowClimbWithoutGold: bool = True):

    def randomLocationExceptOrigin():
      x = random.randint(0, gridWidth-1)
      y = random.randint(0, gridHeight-1)
      if x == 0 and y == 0:
        return randomLocationExceptOrigin()
      else:
        return Coords(x, y)
    
    def pitLocations():
      '''
      loops through each cell on the board (except the origin)
      puts the pit in the cell if the randomly generated
      number exceeds a given probability
      '''
      pitList = []
      for i in range(0, gridWidth):
        for j in range(0, gridHeight):
          if random.random() < pitProb and not (i==0 and j == 0):
           pitList.append(Coords(i,j))
      return pitList
    
    return Environment(gridWidth = gridWidth,
                        gridHeight = gridHeight,
                        pitProb = pitProb,
                        allowClimbWithoutGold = allowClimbWithoutGold,
                        pitLocations = pitLocations(),
                        terminated = False,
                        wumpusLocation = randomLocationExceptOrigin(),
                        wumpusAlive = True,
                        goldLocation = randomLocationExceptOrigin())

In [ ]:
class NaiveAgent():
  def __init__(self):
    pass

  @staticmethod
  def create():
    return Agent()
  
  @staticmethod
  def nextAction():
    actions = {
          0: Forward(),
          1: TurnLeft(),
          2: TurnRight(),
          3: Shoot(),
          4: Grab(),
          5: Climb()
    }
    actions_str = {
          0: 'Forward',
          1: 'TurnLeft',
          2: 'TurnRight',
          3: 'Shoot',
          4: 'Grab',
          5: 'Climb'
    }
    
    randm_int = random.randint(0, 5)
    print(actions_str.get(randm_int))
    return actions.get(randm_int)

In [ ]:
initialEnv = Environment.create()

In [ ]:
agent = NaiveAgent.create()

In [ ]:
percept = Percept.create(initialEnv.isStench(Coords(0,0)), 
                         initialEnv.isBreeze(Coords(0,0)), 
                         initialEnv.isGlitter(Coords(0,0)))

In [ ]:
initialEnv, percept, agent = Forward()(initialEnv, percept, agent)

In [ ]:
initialEnv, percept, agent = TurnLeft()(initialEnv, percept, agent)

In [ ]:
initialEnv, percept, agent = TurnRight()(initialEnv, percept, agent)

In [ ]:
initialEnv.terminated

In [ ]:
initialEnv.visualize(agent)

In [ ]:
def randomLEO():
      x = random.randint(0, 1)
      y = random.randint(0, 1)
      if x == 0 and y == 0:
        return randomLEO()
      else:
        return [x,y]

In [ ]:
print(randomLEO())

In [ ]:
initialEnv.terminated

In [ ]:
a=  Forward()

In [ ]:
c,d = a(initialEnv, percept)

In [ ]:
c.visualize()

In [ ]:
c.terminated

In [ ]:
b = initialEnv.Agent()

In [ ]:
b.forward(4,4)

In [ ]:
initialEnv.wumpusLocation.y

In [ ]:
agent